In [86]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from catboost import CatBoostRegressor

ModuleNotFoundError: No module named 'catboost'

In [21]:
train_data = pd.read_csv('../data/train_data.csv')
test_data = pd.read_csv('../data/sample_submission.csv')
bookings_data = pd.read_csv('../data/bookings_data.csv')
bookings = pd.read_csv('../data/bookings.csv')
hotel_data = pd.read_csv('../data/hotels_data.csv')
customer_data = pd.read_csv('../data/customer_data.csv')
payments_data = pd.read_csv('../data/payments_data.csv')

In [36]:
# keep only entries with payment_sequential as 1
payments_data_unique = payments_data[payments_data['payment_sequential'] == 1]
payments_data_repeat = payments_data[payments_data['payment_sequential'] != 1]

payments_data_unique.describe(include='all')

,booking_id,payment_sequential,payment_type,payment_installments,payment_value
count,99360,99360.0,99360,99360.000000,99360.000000
unique,99360,NaN,5,NaN,NaN
top,6f3fe1789b1e8b2acac839d17b81ef22,NaN,credit_card,NaN,NaN
freq,1,NaN,76476,NaN,NaN
mean,NaN,1.0,NaN,2.929640,158.336774
std,NaN,0.0,NaN,2.714947,220.511857
min,NaN,1.0,NaN,1.000000,0.000000
25%,NaN,1.0,NaN,1.000000,59.947500
50%,NaN,1.0,NaN,2.000000,103.330000
75%,NaN,1.0,NaN,4.000000,175.110000


In [37]:
for payment_data_repeat in payments_data_repeat.itertuples():
    booking_id = payment_data_repeat.booking_id
    payment_value_new = payments_data_unique[payments_data_unique['booking_id'] == booking_id]['payment_value'] + payment_data_repeat.payment_value
    payments_data_unique.loc[payments_data_unique['booking_id'] == booking_id, 'payment_value'] = payment_value_new

payments_data_unique.describe(include='all')

,booking_id,payment_sequential,payment_type,payment_installments,payment_value
count,99360,99360.0,99360,99360.000000,99360.000000
unique,99360,NaN,5,NaN,NaN
top,6f3fe1789b1e8b2acac839d17b81ef22,NaN,credit_card,NaN,NaN
freq,1,NaN,76476,NaN,NaN
mean,NaN,1.0,NaN,2.929640,160.984584
std,NaN,0.0,NaN,2.714947,221.998050
min,NaN,1.0,NaN,1.000000,0.000000
25%,NaN,1.0,NaN,1.000000,62.010000
50%,NaN,1.0,NaN,2.000000,105.290000
75%,NaN,1.0,NaN,4.000000,176.932500


In [22]:
# train_data count
train_data_count = train_data.shape[0]
print('train_data count: ', train_data_count)

# test_data count
test_data_count = test_data.shape[0]
print('test_data count: ', test_data_count)

# bookings description
bookings.describe(include='all')

train_data count:  50000
test_data count:  49079


,booking_id,customer_id,booking_status,booking_create_timestamp,booking_approved_at,booking_checkin_customer_date
count,99441,99441,99441,99441,99281,96476
unique,99441,99441,8,98875,90733,95664
top,c54678b7cc49136f2d6af7e481f51cbd,51297304e76186b10a928d9ef432eb62,completed,2008-04-13 10:31:14,2008-03-01 04:14:10,2008-05-10 23:21:46
freq,1,1,96478,3,9,3


In [23]:
# bookings_data description
bookings_data.describe(include='all')

,booking_id,booking_sequence_id,hotel_id,seller_agent_id,booking_expiry_date,price,agent_fees
count,112650,112650.000000,112650,112650,112650,112650.000000,112650.000000
unique,98666,NaN,32951,3095,93318,NaN,NaN
top,f5f79c56e9e4120aec44ef8272b63d03,NaN,1a7b8ebd4e68314663afaca2eb7d00ea,19b47992c3666cc44a7e94c06560211a,2017-07-21 18:25:23,NaN,NaN
freq,21,NaN,527,2033,21,NaN,NaN
mean,NaN,1.197834,NaN,NaN,NaN,120.653739,19.990320
std,NaN,0.705124,NaN,NaN,NaN,183.633928,15.806405
min,NaN,1.000000,NaN,NaN,NaN,0.850000,0.000000
25%,NaN,1.000000,NaN,NaN,NaN,39.900000,13.080000
50%,NaN,1.000000,NaN,NaN,NaN,74.990000,16.260000
75%,NaN,1.000000,NaN,NaN,NaN,134.900000,21.150000


In [24]:
bookings_data_unique = bookings_data[bookings_data['booking_sequence_id'] == 1]
bookings_data_repeat = bookings_data[bookings_data['booking_sequence_id'] > 1]

bookings_data_unique.describe(include='all')

,booking_id,booking_sequence_id,hotel_id,seller_agent_id,booking_expiry_date,price,agent_fees
count,98666,98666.0,98666,98666,98666,98666.000000,98666.000000
unique,98666,NaN,31881,3088,93001,NaN,NaN
top,242fe8c5a6d1ba2dd792cb1621400010,NaN,856965c36a24e339b605899a4788cb24,19b47992c3666cc44a7e94c06560211a,2017-12-07 04:39:05,NaN,NaN
freq,1,NaN,439,1844,6,NaN,NaN
mean,NaN,1.0,NaN,NaN,NaN,125.964327,20.201927
std,NaN,0.0,NaN,NaN,NaN,191.375106,15.909873
min,NaN,1.0,NaN,NaN,NaN,0.850000,0.000000
25%,NaN,1.0,NaN,NaN,NaN,41.505000,13.310000
50%,NaN,1.0,NaN,NaN,NaN,79.000000,16.360000
75%,NaN,1.0,NaN,NaN,NaN,139.900000,21.230000


In [25]:
for booking_data_repeat in bookings_data_repeat.itertuples():
    bookings_id = booking_data_repeat.booking_id
    booking_data_unique = bookings_data_unique[bookings_data_unique['booking_id'] == bookings_id]
    new_price = booking_data_repeat.price + booking_data_unique.price
    new_agent_fees = booking_data_repeat.agent_fees + booking_data_unique.agent_fees
    bookings_data_unique.loc[bookings_data_unique['booking_id'] == bookings_id, 'price'] = new_price
    bookings_data_unique.loc[bookings_data_unique['booking_id'] == bookings_id, 'agent_fees'] = new_agent_fees

bookings_data_unique.describe(include='all')

,booking_id,booking_sequence_id,hotel_id,seller_agent_id,booking_expiry_date,price,agent_fees
count,98666,98666.0,98666,98666,98666,98666.000000,98666.000000
unique,98666,NaN,31881,3088,93001,NaN,NaN
top,242fe8c5a6d1ba2dd792cb1621400010,NaN,856965c36a24e339b605899a4788cb24,19b47992c3666cc44a7e94c06560211a,2017-12-07 04:39:05,NaN,NaN
freq,1,NaN,439,1844,6,NaN,NaN
mean,NaN,1.0,NaN,NaN,NaN,137.754076,22.823562
std,NaN,0.0,NaN,NaN,NaN,210.645145,21.650909
min,NaN,1.0,NaN,NaN,NaN,0.850000,0.000000
25%,NaN,1.0,NaN,NaN,NaN,45.900000,13.850000
50%,NaN,1.0,NaN,NaN,NaN,86.900000,17.170000
75%,NaN,1.0,NaN,NaN,NaN,149.900000,24.040000


In [33]:
# merge bookings and bookings_data as bookings_df
bookings_df = pd.merge(bookings, bookings_data_unique, on='booking_id', how='left')
bookings_df.describe(include='all')

,booking_id,customer_id,booking_status,booking_create_timestamp,booking_approved_at,booking_checkin_customer_date,booking_sequence_id,hotel_id,seller_agent_id,booking_expiry_date,price,agent_fees
count,99441,99441,99441,99441,99281,96476,98666.0,98666,98666,98666,98666.000000,98666.000000
unique,99441,99441,8,98875,90733,95664,NaN,31881,3088,93001,NaN,NaN
top,c54678b7cc49136f2d6af7e481f51cbd,51297304e76186b10a928d9ef432eb62,completed,2008-04-13 10:31:14,2008-03-01 04:14:10,2008-05-10 23:21:46,NaN,856965c36a24e339b605899a4788cb24,19b47992c3666cc44a7e94c06560211a,2018-06-11 03:31:04,NaN,NaN
freq,1,1,96478,3,9,3,NaN,439,1844,6,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,137.754076,22.823562
std,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,210.645145,21.650909
min,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.850000,0.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,45.900000,13.850000
50%,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,86.900000,17.170000
75%,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,149.900000,24.040000


In [63]:
# merge bookings_df and customer_data as bookings_customer_df
bookings_customer_df = pd.merge(bookings_df, customer_data, on='customer_id', how='left')

# merge bookings_customer_merged and hotel_data as bookings_hotel_merged
bookings_hotel_df = pd.merge(bookings_customer_df, hotel_data, on='hotel_id', how='left')

bookings_hotel_df.drop(['customer_id', 'hotel_id', 'booking_sequence_id'], axis=1, inplace=True)

bookings_hotel_df.describe(include='all')

,booking_id,booking_status,booking_create_timestamp,booking_approved_at,booking_checkin_customer_date,seller_agent_id,booking_expiry_date,price,agent_fees,customer_unique_id,country,hotel_category,hotel_name_length,hotel_description_length,hotel_photos_qty
count,99441,99441,99441,99281,96476,98666,98666,98666.000000,98666.000000,99441,99441,97250.000000,97250.000000,97250.000000,97250.000000
unique,99441,8,98875,90733,95664,3088,93001,NaN,NaN,96096,9,NaN,NaN,NaN,NaN
top,c54678b7cc49136f2d6af7e481f51cbd,completed,2008-04-13 10:31:14,2008-03-01 04:14:10,2008-05-10 23:21:46,19b47992c3666cc44a7e94c06560211a,2018-06-11 03:31:04,NaN,NaN,f50201ccdcedfb9e2ac84558d50f5ead,Slovakia,NaN,NaN,NaN,NaN
freq,1,96478,3,9,3,1844,6,NaN,NaN,17,11212,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,137.754076,22.823562,NaN,NaN,28.856422,48.846386,794.161398,2.250591
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,210.645145,21.650909,NaN,NaN,22.666001,9.999239,654.751953,1.747095
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.850000,0.000000,NaN,NaN,1.000000,5.000000,4.000000,1.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.900000,13.850000,NaN,NaN,10.000000,42.000000,349.000000,1.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,86.900000,17.170000,NaN,NaN,28.000000,52.000000,607.000000,2.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,149.900000,24.040000,NaN,NaN,38.000000,57.000000,996.000000,3.000000


In [83]:
bookings_payment_df = pd.merge(bookings_hotel_df, payments_data_unique, on='booking_id', how='left')

bookings_payment_df.drop(['payment_sequential'], axis=1, inplace=True)

# convert seller_agent_id to integers based on unique values
bookings_payment_df['seller_agent_id'] = bookings_payment_df['seller_agent_id'].astype('category')
bookings_payment_df['seller_agent_id'] = bookings_payment_df['seller_agent_id'].cat.codes

# convert booking_status to integers based on unique values
bookings_payment_df['booking_status'] = bookings_payment_df['booking_status'].astype('category')
bookings_payment_df['booking_status'] = bookings_payment_df['booking_status'].cat.codes

# convert country to integers based on unique values
bookings_payment_df['country'] = bookings_payment_df['country'].astype('category')
bookings_payment_df['country'] = bookings_payment_df['country'].cat.codes

# convert payment_type to integers based on unique values
bookings_payment_df['payment_type'] = bookings_payment_df['payment_type'].astype('category')
bookings_payment_df['payment_type'] = bookings_payment_df['payment_type'].cat.codes

# convert customer unique id to integers based on unique values
bookings_payment_df['customer_unique_id'] = bookings_payment_df['customer_unique_id'].astype('category')
bookings_payment_df['customer_unique_id'] = bookings_payment_df['customer_unique_id'].cat.codes

print(bookings_payment_df.dtypes)
bookings_payment_df.describe()

booking_id                        object
booking_status                      int8
booking_create_timestamp          object
booking_approved_at               object
booking_checkin_customer_date     object
seller_agent_id                    int16
booking_expiry_date               object
price                            float64
agent_fees                       float64
customer_unique_id                 int32
country                             int8
hotel_category                   float64
hotel_name_length                float64
hotel_description_length         float64
hotel_photos_qty                 float64
payment_type                        int8
payment_installments             float64
payment_value                    float64
dtype: object


,booking_status,seller_agent_id,price,agent_fees,customer_unique_id,country,hotel_category,hotel_name_length,hotel_description_length,hotel_photos_qty,payment_type,payment_installments,payment_value
count,99441.000000,99441.000000,98666.000000,98666.000000,99441.000000,99441.000000,97250.000000,97250.000000,97250.000000,97250.000000,99441.000000,99360.000000,99360.000000
mean,2.076166,1393.214670,137.754076,22.823562,48049.895224,4.011484,28.856422,48.846386,794.161398,2.250591,0.477218,2.929640,160.984584
std,0.561226,932.046188,210.645145,21.650909,27758.278975,2.580036,22.666001,9.999239,654.751953,1.747095,0.919129,2.714947,221.998050
min,0.000000,-1.000000,0.850000,0.000000,0.000000,0.000000,1.000000,5.000000,4.000000,1.000000,-1.000000,1.000000,0.000000
25%,2.000000,499.000000,45.900000,13.850000,23986.000000,2.000000,10.000000,42.000000,349.000000,1.000000,0.000000,1.000000,62.010000
50%,2.000000,1318.000000,86.900000,17.170000,48053.000000,4.000000,28.000000,52.000000,607.000000,2.000000,0.000000,2.000000,105.290000
75%,2.000000,2235.000000,149.900000,24.040000,72088.000000,6.000000,38.000000,57.000000,996.000000,3.000000,0.000000,4.000000,176.932500
max,7.000000,3087.000000,13440.000000,1794.960000,96095.000000,8.000000,73.000000,76.000000,3992.000000,20.000000,4.000000,24.000000,13664.080000


In [84]:
date_columns = ['booking_create_timestamp', 'booking_approved_at', 'booking_checkin_customer_date','booking_expiry_date']

base_date = pd.to_datetime('2007-06-01')
base_minutes = base_date.value / 10**9 / 60
for date_column in date_columns:
    bookings_payment_df[date_column] = pd.to_datetime(bookings_payment_df[date_column])
    bookings_payment_df[date_column] = bookings_payment_df[date_column].apply(lambda x: (x.value / 10**9 / 60) - base_minutes)

print(bookings_payment_df.dtypes)
bookings_payment_df.describe(include='all')

booking_id                        object
booking_status                      int8
booking_create_timestamp         float64
booking_approved_at              float64
booking_checkin_customer_date    float64
seller_agent_id                    int16
booking_expiry_date              float64
price                            float64
agent_fees                       float64
customer_unique_id                 int32
country                             int8
hotel_category                   float64
hotel_name_length                float64
hotel_description_length         float64
hotel_photos_qty                 float64
payment_type                        int8
payment_installments             float64
payment_value                    float64
dtype: object


,booking_id,booking_status,booking_create_timestamp,booking_approved_at,booking_checkin_customer_date,seller_agent_id,booking_expiry_date,price,agent_fees,customer_unique_id,country,hotel_category,hotel_name_length,hotel_description_length,hotel_photos_qty,payment_type,payment_installments,payment_value
count,99441,99441.000000,99441.000000,9.944100e+04,9.944100e+04,99441.000000,9.944100e+04,98666.000000,98666.000000,99441.000000,99441.000000,97250.000000,97250.000000,97250.000000,97250.000000,99441.000000,99360.000000,99360.000000
unique,99441,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,c54678b7cc49136f2d6af7e481f51cbd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,2.076166,311546.212943,3.263581e+04,-4.848210e+06,1393.214670,4.183003e+06,137.754076,22.823562,48049.895224,4.011484,28.856422,48.846386,794.161398,2.250591,0.477218,2.929640,160.984584
std,NaN,0.561226,221283.646423,6.965937e+06,2.954956e+07,932.046188,1.574034e+07,210.645145,21.650909,27758.278975,2.580036,22.666001,9.999239,654.751953,1.747095,0.919129,2.714947,221.998050
min,NaN,0.000000,-383221.683333,-1.734005e+08,-1.734005e+08,-1.000000,-1.734005e+08,0.850000,0.000000,0.000000,0.000000,1.000000,5.000000,4.000000,1.000000,-1.000000,1.000000,0.000000
25%,NaN,2.000000,153509.316667,1.532087e+05,1.522746e+05,499.000000,5.413723e+06,45.900000,13.850000,23986.000000,2.000000,10.000000,42.000000,349.000000,1.000000,0.000000,1.000000,62.010000
50%,NaN,2.000000,338327.600000,3.384823e+05,3.507904e+05,1318.000000,5.603588e+06,86.900000,17.170000,48053.000000,4.000000,28.000000,52.000000,607.000000,2.000000,0.000000,2.000000,105.290000
75%,NaN,2.000000,490525.266667,4.906134e+05,5.048219e+05,2235.000000,5.754815e+06,149.900000,24.040000,72088.000000,6.000000,38.000000,57.000000,996.000000,3.000000,0.000000,4.000000,176.932500


In [85]:
# train_booking_df contains bookings_df with booking_id in train_data
train_booking_df = bookings_payment_df[bookings_payment_df['booking_id'].isin(train_data['booking_id'])]

,booking_id,booking_status,booking_create_timestamp,booking_approved_at,booking_checkin_customer_date,seller_agent_id,booking_expiry_date,price,agent_fees,customer_unique_id,country,hotel_category,hotel_name_length,hotel_description_length,hotel_photos_qty,payment_type,payment_installments,payment_value
count,49868,49868.000000,49868.000000,4.986800e+04,4.986800e+04,49868.000000,4.986800e+04,49480.000000,49480.000000,49868.000000,49868.000000,48786.000000,48786.000000,48786.000000,48786.000000,49868.000000,49824.000000,49824.000000
unique,49868,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,d0c44946d9cf07ec65d47770eb9100c2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,2.075098,312787.583979,6.256771e+04,-4.715002e+06,1392.563548,4.186598e+06,136.783206,22.859228,47968.966672,4.000401,28.795597,48.861907,793.158898,2.247551,0.473350,2.920259,159.940446
std,NaN,0.559453,221178.476425,6.599690e+06,2.918235e+07,931.671271,1.572747e+07,213.349405,22.249972,27772.637565,2.582794,22.630239,10.001610,651.296521,1.744223,0.912956,2.709641,224.018773
min,NaN,0.000000,-342849.133333,-1.734005e+08,-1.734005e+08,-1.000000,-1.734005e+08,0.850000,0.000000,1.000000,0.000000,1.000000,5.000000,4.000000,1.000000,-1.000000,1.000000,0.000000
25%,NaN,2.000000,154973.091667,1.550649e+05,1.537957e+05,505.000000,5.417574e+06,45.900000,13.830000,23717.500000,2.000000,10.000000,43.000000,349.000000,1.000000,0.000000,1.000000,61.800000
50%,NaN,2.000000,339132.575000,3.392281e+05,3.539749e+05,1318.000000,5.603856e+06,85.990000,17.150000,47974.500000,4.000000,28.000000,52.000000,607.000000,2.000000,0.000000,2.000000,105.000000
75%,NaN,2.000000,492028.670833,4.920533e+05,5.051857e+05,2235.000000,5.755213e+06,149.900000,24.112500,72046.250000,6.000000,38.000000,57.000000,995.000000,3.000000,0.000000,4.000000,175.992500
